In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ["DATA_ROOT"] = "/content/drive/MyDrive/hazard-data"

In [ ]:
import sys
sys.path.append("..")

from src.utils import load_config, get_output_dir, ensure_dir
from src.gee_pipeline import init_ee, get_s2_collection, load_boundary_fc, load_classes, make_class_mask
from src.tiling import generate_tiles
from src.export import get_first_tile_url

cfg = load_config("../configs/config.yaml")
out_dir = get_output_dir(cfg)
ensure_dir(out_dir)